In [2]:
import numpy as np
import pandas as pd
from citipy import citipy



In [3]:
lats = np.random.uniform(low= -90.000, high= 90.000, size= 2000)
lngs = np.random.uniform(low= -90.000, high= 90.000, size= 2000)


In [4]:
lat_lngs = zip(lats,lngs)

In [5]:
# Make a list out of the lat_lng object
coordinates = list(lat_lngs)

In [6]:
#find the city closest to the coordinate, and put them in a list
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

len(cities)

764

In [7]:
# Make API calls based on the city name in the cities list. API doc url:https://openweathermap.org/current
# Import dependencies
import requests
from config import weather_api_key
from datetime import datetime
city_data = []


# Set up URL for request, *** UNIT IS IN METRIC ***
url = "http://api.openweathermap.org/data/2.5/weather?units=metric&APPID=" + weather_api_key

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

record_count = 1
set_count = 1

# Loop through cities list and request the weather data
for i,city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Description":city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jacareacanga
Processing Record 2 of Set 1 | adrar
Processing Record 3 of Set 1 | santa isabel do rio negro
Processing Record 4 of Set 1 | great yarmouth
Processing Record 5 of Set 1 | comodoro rivadavia
Processing Record 6 of Set 1 | cidreira
Processing Record 7 of Set 1 | manyana
Processing Record 8 of Set 1 | lebu
Processing Record 9 of Set 1 | jalu
Processing Record 10 of Set 1 | faaborg
Processing Record 11 of Set 1 | iqaluit
Processing Record 12 of Set 1 | svetlogorsk
Processing Record 13 of Set 1 | namibe
Processing Record 14 of Set 1 | georgetown
Processing Record 15 of Set 1 | ushtobe
Processing Record 16 of Set 1 | raudeberg
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | cape town
Processing Record 19 of Set 1 | maloshuyka
City not found. Skipping...
Processing Record 20 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 21 of Set 1 | ostrovnoy


In [8]:
# Convert city_data into a dataframe
city_data_df = pd.DataFrame(city_data)

# Reorder the dataframe
new_column_order = ["City","Cloudiness","Country","Date","Humidity","Lat","Lng","Max Temp","Wind Speed","Description"]
city_data_df[new_column_order]

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Description
0,Jacareacanga,0,BR,2021-08-24 18:51:10,46,-6.2222,-57.7528,36.17,3.09,clear sky
1,Adrar,3,MR,2021-08-24 18:50:41,14,20.5022,-10.0711,38.78,2.60,clear sky
2,Santa Isabel Do Rio Negro,89,BR,2021-08-24 18:48:53,33,-0.4139,-65.0192,36.26,2.24,overcast clouds
3,Great Yarmouth,53,GB,2021-08-24 18:54:14,96,52.6083,1.7305,17.53,4.46,broken clouds
4,Comodoro Rivadavia,0,AR,2021-08-24 18:57:18,46,-45.8667,-67.5000,9.92,8.23,clear sky
...,...,...,...,...,...,...,...,...,...,...
704,Urengoy,16,RU,2021-08-24 19:00:14,76,65.9667,78.3667,11.20,4.17,few clouds
705,Ashqelon,0,IL,2021-08-24 19:00:14,66,31.6693,34.5715,28.23,2.99,clear sky
706,Taoudenni,0,ML,2021-08-24 19:00:15,7,22.6783,-3.9836,40.88,4.89,clear sky
707,Samana,100,DO,2021-08-24 19:00:15,67,19.2056,-69.3369,29.38,3.54,overcast clouds


In [9]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")